In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

## 3D meshing example

This notebook shows how to mesh a 3D volume:

1. Load and visualize a volume
1. Apply image filters and segment image
1. Generate a 3D surface mesh from the binary image
1. Decimate/simplify a 3D mesh
1. Visualize and save the mesh to gmsh22 format

In [ ]:
from nanomesh.volume import Volume
import pyvista as pv
from skimage import filters
import numpy as np
from skimage.transform import rescale

Load the data and select a subvolume to work with for this notebook.

In [ ]:
full_vol = Volume.load('slab_x450-550.npy')

# down/subsample to keep cpu times in check
scaled = full_vol.select_subvolume(ys=(400,500), zs=(600,800)).apply(rescale, scale=0.25)
scaled.show_slice()

In [ ]:
vol_gauss = scaled.apply(filters.gaussian, sigma=1)

thresh = vol_gauss.apply(filters.threshold_yen)

seg_vol = vol_gauss.apply(np.digitize, bins=[thresh])

seg_vol.show_slice()

### Generate 3d tetragonal mesh

Meshes can be generated using the `Mesher3D` class.

A convenience method is also available through `volume.generate_mesh`, which offers less flexibility.

In [ ]:
%%time

from nanomesh.mesh3d import Mesher3D
from nanomesh.mesh_utils import VolumeMeshContainer

mesher = Mesher3D(seg_vol.image)
mesher.generate_contour(label=0, smooth=True)
mesher.generate_contour(label=1, smooth=True)

vmesh = mesher.tetrahedralize(label=1, order=1, mindihedral=30, minratio=1.1, verbose=2)

vmesh.plot_itk()

In [ ]:
vmesh.plot_submesh(color='orange')

Save the data.

In [ ]:
vmeshio = vmesh.to_meshio().write('volume_mesh.msh', file_format='gmsh22', binary=False)